In [110]:
from xpinyin import Pinyin
import pandas as pd
from keras.layers import LSTM
from keras.models import Sequential
import numpy as np

In [100]:
p = Pinyin()
df = pd.read_csv('ChineseNames.csv')

In [101]:
#change headers to english
df.columns = ['Name', 'Gender']

In [102]:
#strip last name
def strip_last_name(name):
    return name[1:]

def convert_to_pinyin(name):
    return p.get_pinyin(name)

def process_name(name):
    name = strip_last_name(name)
    name = convert_to_pinyin(name)
    return name

In [103]:
#visualize before after processing
print(df.tail(5))
df['Name'] = df['Name'].apply(process_name)
print(df.tail(5))

     Name Gender
9777   邹敏      F
9778  邹泞憶      F
9779  邹莎莎      F
9780  邹苇航      F
9781  邹雪飞      F
9782  邹雅文      F
9783  邹颜鞠      F
9784  邹一鑫      F
9785  邹逸宁      F
9786   邹莹      F
9787  邹滢曼      F
9788  邹雨宏      F
9789  邹雨寰      F
9790  邹玉洁      F
9791   左曈      F
9792  左婉怡      F
9793  左烜晅      F
9794  左雨晴      F
9795   左越      F
9796  左子烨      F
           Name Gender
9777        min      F
9778    ning-yi      F
9779    sha-sha      F
9780   wei-hang      F
9781    xue-fei      F
9782     ya-wen      F
9783     yan-ju      F
9784     yi-xin      F
9785    yi-ning      F
9786       ying      F
9787   ying-man      F
9788    yu-hong      F
9789    yu-huan      F
9790      yu-ji      F
9791       tong      F
9792     wan-yi      F
9793  xuan-xuan      F
9794    yu-qing      F
9795        yue      F
9796      zi-ye      F


In [104]:
#get all unique chars
characters = {}
for i, name in enumerate(df['Name']):
    for char in name:
        if char in characters:
            characters[char] += 1
        else:
            characters[char] = 1

In [105]:
#find the longest name
maxlen = df['Name'].str.len().max()

In [106]:
#create mapping.  though it would be nice if they mapping is alphabetically ordered, but it doesn't matter mathematically.
idx = 0
for c in characters.keys():
    characters[c] = idx
    idx += 1

In [210]:
#create training example of dimension (example #, timestep, # of features (length of 1-hot vector))
num_examples = df.shape[0]
time_steps = maxlen
num_features = len(characters)
def char_to_vec(char):
    vector = np.zeros((1, num_features), dtype=int)
    idx = characters[char]
    vector[0,idx] = 1
    return vector

def name_to_vec(name):
    example = np.zeros((maxlen, num_features), dtype = int)
    for i in range(maxlen):
        if i < len(name):
            char = name[i]
            vector = char_to_vec(char)
            example[i,:] = vector[0,:]
        else:
            example[i,:] = np.zeros((num_features),dtype=int)
    return example

In [259]:
#convert all examples to vector format
X = np.empty((df.shape[0], maxlen, num_features))
for i, name in enumerate(df['Name']):
    example = name_to_vec(name)
    X[i,:,:] = example

In [266]:
Y = df['Gender'].values
Y[Y == 'M'] = 1
Y[Y == 'F'] = 0

In [269]:
model = Sequential()
model.add(LSTM(1, input_shape=(maxlen, num_features), return_sequences=False))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [270]:
model.fit(X,Y, batch_size=32, epochs=10, verbose=True)

Epoch 1/10
9797/9797 [==============================] - 4s 382us/step - loss: 1.7560 - acc: 0.5001
Epoch 2/10
9797/9797 [==============================] - 3s 282us/step - loss: 0.8483 - acc: 0.5124
Epoch 3/10
9797/9797 [==============================] - 3s 288us/step - loss: 0.6815 - acc: 0.5708
Epoch 4/10
9797/9797 [==============================] - 3s 291us/step - loss: 0.6761 - acc: 0.5852
Epoch 5/10
9797/9797 [==============================] - 3s 297us/step - loss: 0.6700 - acc: 0.6006
Epoch 6/10
9797/9797 [==============================] - 3s 304us/step - loss: 0.6643 - acc: 0.6080
Epoch 7/10
9797/9797 [==============================] - 3s 305us/step - loss: 0.6599 - acc: 0.6168
Epoch 8/10
9797/9797 [==============================] - 3s 303us/step - loss: 0.6565 - acc: 0.6170
Epoch 9/10
9797/9797 [==============================] - 3s 306us/step - loss: 0.6537 - acc: 0.6217
Epoch 10/10
9797/9797 [==============================] - 3s 308us/step - loss: 0.6507 - acc: 0.6243


In [280]:
model.predict_classes(np.expand_dims(name_to_vec('chu'), 0))

array([[1]])